# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Shi  ->  J. Shi  |  ['J. Shi']
T. Henning  ->  T. Henning  |  ['T. Henning']
J. Eberhardt  ->  J. Eberhardt  |  ['J. Eberhardt']
A. d. Graaff  ->  A. D. Graaff  |  ['A. D. Graaff']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
C. Clontz  ->  C. Clontz  |  ['C. Clontz']


S. Joharle  ->  S. Joharle  |  ['S. Joharle']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 82 new papers today
          10 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/10 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.00063


extracting tarball to tmp_2510.00063...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.00102


extracting tarball to tmp_2510.00102...

...

Found 104 bibliographic references in tmp_2510.00330/paper.bbl.
Retrieving document from  https://arxiv.org/e-print/2510.00897


extracting tarball to tmp_2510.00897... done.
Retrieving document from  https://arxiv.org/e-print/2510.00945


extracting tarball to tmp_2510.00945...

 done.


Found 150 bibliographic references in tmp_2510.00945/blank.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.00102-b31b1b.svg)](https://arxiv.org/abs/2510.00102) | **A Cold and Super-Puffy Planet on a Polar Orbit**  |
|| J. I. Espinoza-Retamal, et al. -- incl., <mark>T. Henning</mark>, <mark>J. Eberhardt</mark> |
|*Appeared on*| *2025-10-02*|
|*Comments*| *16 pages, 4 figures, submitted to AAS journals*|
|**Abstract**|            We report the discovery of TOI-4507 b, a transiting sub-Saturn with a density $<0.3$ g/cm$^3$ on a 105-day polar orbit around a $700$ Myr old F star. The transits were detected using data from TESS as well as the Antarctic telescope ASTEP. A joint analysis of the light curves and radial velocities from HARPS, FEROS, and CORALIE confirmed the planetary nature of the signal by limiting the mass to be below $30\,M_\oplus$ at $95\%$ confidence. The radial velocities also exhibit the Rossiter-McLaughlin effect and imply that the star's equatorial plane is tilted by $82.0_{-2.4}^{+2.6}$ deg with respect to the planet's orbital plane. With these characteristics, TOI-4507 b is one of longest-period planets for which the stellar obliquity has been measured, and is among the longest-period and youngest ''super-puff'' planets yet discovered.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.00103-b31b1b.svg)](https://arxiv.org/abs/2510.00103) | **The warm outer layer of a Little Red Dot as the source of [Fe II] and collisional Balmer lines with scattering wings**  |
|| A. Torralba, et al. -- incl., <mark>A. d. Graaff</mark> |
|*Appeared on*| *2025-10-02*|
|*Comments*| *18 pages incl. references. Main figure: 4 ([Fe II] model). To be submitted to A&A. Comments welcome*|
|**Abstract**|            The population of the Little Red Dots (LRDs) may represent a key phase of supermassive black hole (SMBH) growth. A cocoon of dense excited gas is emerging as key component to explain the most striking properties of LRDs, such as strong Balmer breaks and Balmer absorption, as well as the weak IR emission. To dissect the structure of LRDs, we analyze new deep JWST/NIRSpec PRISM and G395H spectra of FRESCO-GN-9771, one of the most luminous known LRDs at $z=5.5$. These reveal a strong Balmer break, broad Balmer lines and very narrow [O III] emission. We unveil a forest of optical [Fe II] lines, which we argue is emerging from a dense ($n_{\rm H}=10^{9-10}$ cm$^{-3}$) warm layer with electron temperature $T_{\rm e}\approx7000$ K. The broad wings of H$\alpha$ and H$\beta$ have an exponential profile due to electron scattering in this same layer. The high $\rm H\alpha:H\beta:H\gamma$ flux ratio of $\approx10.4:1:0.14$ is an indicator of collisional excitation and resonant scattering dominating the Balmer line emission. A narrow H$\gamma$ component, unseen in the other two Balmer lines due to outshining by the broad components, could trace the ISM of a normal host galaxy with a star formation rate $\sim5$ M$_{\odot}$ yr$^{-1}$. The warm layer is mostly opaque to Balmer transitions, producing a characteristic P-Cygni profile in the line centers suggesting outflowing motions. This same layer is responsible for shaping the Balmer break. The broad-band spectrum can be reasonably matched by a simple photoionized slab model that dominates the $\lambda>1500$ Å continuum and a low mass ($\sim10^8$ M$_{\odot}$) galaxy that could explain the narrow [O III], with only subdominant contribution to the UV continuum. Our findings indicate that Balmer lines are not directly tracing gas kinematics near the SMBH and that the BH mass scale is likely much lower than virial indicators suggest.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.00299-b31b1b.svg)](https://arxiv.org/abs/2510.00299) | **A planetary system with a sub-Neptune planet in the habitable zone of TOI-2093**  |
|| J. Sanz-Forcada, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2025-10-02*|
|*Comments*| *Accepted by A&A. 12 pages, 7 figures, 5 tables in the main body (20 pages, 19 figures, 9 tables including appendices)*|
|**Abstract**|            Aims. We aim to confirm and measure the mass of the transiting planet candidate around the K5V star TOI-2093, previously announced by the Transiting Exoplanet Survey Satellite (TESS) project. Methods. We combined photometric data from 32 sectors between 2019 and 2024 with 86 radial velocity measurements obtained with the CARMENES spectrograph over a period of 2.4 years, along with a series of ground-based, broadband photometric monitoring campaigns to characterize the host star and the transiting planet candidate, as well as to search for additional planets in the system. Our data indicate that TOI-2093 is a main-sequence star located at a distance of 83 pc, with solar metallicity, and a rotation period of 43.8 +- 1.8 d. Results. We have confirmed the planetary nature of the TESS transiting planet candidate, named TOI-2093 c, through the detection of its Keplerian signal in the spectroscopic data. We measured a planetary radius of 2.30 +- 0.12 Rearth, a Neptune-like mass of 15.8 +- 3.7 Mearth, and an orbital period of 53.81149 +- 0.00017 d. This makes TOI-2093 c the smallest exoplanet known in the habitable zone of a main-sequence FGK star. Given its size and relatively high density, TOI-2093 c belongs to a class of planets with no analog in the Solar System. In addition, the CARMENES data revealed the presence of a second planet candidate with a minimum mass of 10.6 +- 2.5 Mearth and an orbital period of 12.836 +- 0.021 d. This inner planet, which we designated TOI-2093 b, shows no detectable photometric transit in the TESS light curves. The orbital planes of the two planets are misaligned by more than 1.6 deg despite the near 4:1 mean-motion resonance of their orbital periods.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.00330-b31b1b.svg)](https://arxiv.org/abs/2510.00330) | **oMEGACat. VII. Tracing Interstellar and Intracluster Medium of $ω$ Centauri using Sodium Absorptions**  |
|| Z. Wang, et al. -- incl., <mark>M. Häberle</mark>, <mark>N. Neumayer</mark>, <mark>C. Clontz</mark> |
|*Appeared on*| *2025-10-02*|
|*Comments*| *23 pages, 11 figures, and 2 tables, accepted by ApJ. Machine-readable data is available in the online article*|
|**Abstract**|            We investigate the foreground interstellar medium along the line of sight and intracluster medium of $\omega$ Centauri ($\omega$ Cen) by measuring the equivalent width of Na I D absorptions from MUSE observations. The large line-of-sight velocity difference between $\omega$ Cen and the foreground enables us to separate Na I D absorption contributed from atomic gas in the interstellar and intracluster medium. We find that small-scale substructures in the foreground Na I D distribution correlate with differential reddening derived from photometric methods. Using an empirical Na I D equivalent width-reddening relation, we determine an average reddening of $E(B-V)=0.153\pm0.003$ mag within the half-light radius of $\omega$ Cen. However, the Na I D-inferred differential reddening is significantly larger than photometric estimates. This is likely due to scatter in the Na I D-reddening relation. We find no evidence for intracluster atomic gas from spectra of horizontal branch stars, as there is no significant Na I D absorption at $\omega$ Cen's systemic velocity. Given this non-detection, we place the strongest upper limit to date on the intracluster atomic gas column density in $\omega$ Cen of $\lesssim2.17 \times 10^{18}~\rm{cm^{-2}}$. We also estimate the ionized gas density from pulsar dispersion measure variations, which exceed the atomic gas limit by $\sim$50 times. Nevertheless, the strong correlation between dispersion measure and foreground Na I D suggests that much or all of this ionized gas resides in the foreground. Given ongoing mass loss from bright giant stars, our findings imply that the intracluster gas accumulation timescale is short, and gas removal in the cluster is likely not tied to stripping as $\omega$ Cen passes through the Galactic disk.         |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.00271-b31b1b.svg)](https://arxiv.org/abs/2510.00271) | **Searching for Ultra-light Dark Matter in Spatial Correlations of White Dwarf Structure**  |
|| N. R. Crumpler, N. L. Zakamska, G. A. Pallathadka, <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-10-02*|
|*Comments*| *24 pages, 9 figures*|
|**Abstract**|            If dark matter is ultra-light and has certain Standard Model interactions, it can change the mass-radius relation of white dwarf stars. The coherence length of ultra-light dark matter imparts spatial correlations in deviations from the canonical mass-radius relation, and thus white dwarfs can be used to reconstruct the coherence length, or equivalently the particle mass, of the dark matter field. We simulate the observability of such spatial correlations accounting for realistic complications like variable hydrogen envelope thickness, dust, binaries, measurement noise, and distance uncertainties in DA white dwarfs. Using a machine learning approach on simulated data, we measure the dark matter field coherence length and find that large deviations from the mass-radius relation ($\sim10\%$ change in radius) are needed to produce an observable signal given realistic noise sources. We apply our spatial correlation measurement routine to the SDSS catalog of 10,207 DA white dwarfs. We detect a positive spatial correlation among white dwarfs at separations corresponding to a coherence length of $300\pm50$ pc, with an average Z-score of 85 for white dwarfs separated by less than this coherence length. We conclude that this signal is due to observational bias. The signal can be explained by an offset between measurements and theory for nearby cool white dwarfs, and the presence of few, low-temperature white dwarfs with noisy measurements at further distances. With future improvements in white dwarf models and measurement techniques, particularly for cool white dwarfs, this method can provide interesting constraints on ultra-light dark matter models.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.00063-b31b1b.svg)](https://arxiv.org/abs/2510.00063) | **AstroMMBench: A Benchmark for Evaluating Multimodal Large Language Models Capabilities in Astronomy**  |
|| <mark>J. Shi</mark>, et al. |
|*Appeared on*| *2025-10-02*|
|*Comments*| **|
|**Abstract**|            Astronomical image interpretation presents a significant challenge for applying multimodal large language models (MLLMs) to specialized scientific tasks. Existing benchmarks focus on general multimodal capabilities but fail to capture the complexity of astronomical data. To bridge this gap, we introduce AstroMMBench, the first comprehensive benchmark designed to evaluate MLLMs in astronomical image understanding. AstroMMBench comprises 621 multiple-choice questions across six astrophysical subfields, curated and reviewed by 15 domain experts for quality and relevance. We conducted an extensive evaluation of 25 diverse MLLMs, including 22 open-source and 3 closed-source models, using AstroMMBench. The results show that Ovis2-34B achieved the highest overall accuracy (70.5%), demonstrating leading capabilities even compared to strong closed-source models. Performance showed variations across the six astrophysical subfields, proving particularly challenging in domains like cosmology and high-energy astrophysics, while models performed relatively better in others, such as instrumentation and solar astrophysics. These findings underscore the vital role of domain-specific benchmarks like AstroMMBench in critically evaluating MLLM performance and guiding their targeted development for scientific applications. AstroMMBench provides a foundational resource and a dynamic tool to catalyze advancements at the intersection of AI and astronomy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.00135-b31b1b.svg)](https://arxiv.org/abs/2510.00135) | **SN 2025coe: A Triple-Peaked Calcium-Strong Transient from A White-Dwarf Progenitor**  |
|| C. Chen, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-02*|
|*Comments*| *12 pages, 9 figures, submitted to ApJ*|
|**Abstract**|            SN 2025coe is a calcium-strong transient located at an extremely large projected offset $\sim$39.3 kpc from the center of its host, the nearby early-type galaxy NGC 3277 at a distance of $\sim$25.5 Mpc. In this paper, we present multi-band photometric and spectroscopic observations spanning $\sim$100 days post-discovery. Its multi-band light curves display three distinct peaks: (1) an initial peak at $t \approx 1.6$ days attributed to shock cooling emission, (2) a secondary peak of $M_{R, \, peak} \approx$ $-$15.8 mag at $t \approx 10.2$ days powered by radioactive decay, and (3) a late-time bump at $t \approx 42.8$ days likely caused by ejecta-circumstellar material/clump interaction. Spectral evolution of SN 2025coe reveals a fast transition to the nebular phase within 2 months, where it exhibits an exceptionally high [Ca II]/[O I] ratio larger than 6. Modeling of the bolometric light curve suggests an ejecta mass of $M_{\rm ej} = 0.29^{+0.14}_{-0.15} \, M_{\odot}$, a $^{56}$Ni mass of $M_{\rm ^{56}Ni} = 2.4^{+0.06}_{-0.05} \times 10^{-2} M_{\odot}$, and a progenitor envelope with mass $M_e = 1.4^{+6.9}_{-1.2} \times 10^{-3} \, M_{\odot}$ and radius $R_e = 13.5^{+64.1}_{-11.1} \, R_{\odot}$. The tidal disruption of a hybrid HeCO white dwarf (WD) by a low-mass CO WD provides a natural explanation for the low ejecta mass, the small fraction of $^{56}$Ni, and the presence of an extended, low-mass envelope.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.00176-b31b1b.svg)](https://arxiv.org/abs/2510.00176) | **The Simons Observatory: Characterization of All DC/RF Routing Wafers for Detector Modules**  |
|| A. Middleton, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-02*|
|*Comments*| *5 pages, 7 figures. Submitted to LTD 2025 conference proceedings*|
|**Abstract**|            The Simons Observatory (SO) is a cosmic microwave background experiment with over 67,000 polarization-sensitive transition-edge sensor (TES) detectors currently installed for use in observations and plans to increase the total detector count to ${\sim}$98,000 detectors with the Advanced SO upgrade. The TES arrays are packaged into Universal Focal-Plane Modules (UFMs), which also contain the multiplexing readout circuit. Within a readout module, a DC/RF routing wafer provides a cold interface between the detectors and the readout multiplexing chips. Each routing wafer hosts twelve bias lines, which contain the ${\sim}$400 ${\mu}{\Omega}$ shunt resistors that are part of the TES bias circuitry. More than 70 routing wafers have been fabricated and tested both at room temperature and 100 mK before integration into UFMs. The lab measurements for all screened wafers have been compiled to show the distribution of measured average shunt resistance Rsh for each bias line, both across bias lines on a single routing wafer and across all routing wafers. The mean average shunt resistance for all wafers was found to be 396 ${\mu}{\Omega}$ with a standard deviation of 16 ${\mu}{\Omega}$, or ${\sim}$4%. For each wafer, we note good uniformity of average Rsh between bias lines, with a slight downward trend with increasing distance from the center of the wafer. The fabrication data collected at room temperature shows agreement with the cryogenic measurements of Rsh distribution.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.00897-b31b1b.svg)](https://arxiv.org/abs/2510.00897) | **The gas streamer G1-2-3 in the Galactic Center**  |
|| S. Gillessen, et al. -- incl., <mark>S. Joharle</mark> |
|*Appeared on*| *2025-10-02*|
|*Comments*| *7 pages, 5 figures, under review at A&A*|
|**Abstract**|            The black hole in the Galactic Center, Sgr A*, is prototypical for ultra-low-fed galactic nuclei. The discovery of a hand-full of gas clumps in the realm of a few Earth masses in its immediate vicinity provides a gas reservoir sufficient to power Sgr A*. In particular, the gas cloud G2 is of interest due to its extreme orbit, on which it passed at a pericenter distance of around 100 AU and notably lost kinetic energy during the fly-by due to the interaction with the black hole accretion flow. 13 years prior to G2, a resembling gas cloud called G1, passed Sgr A* on a similar orbit. The origin of G2 remained a topic of discussion, with models including a central (stellar) source still proposed as alternatives to pure gaseous clouds. Here, we report the orbit of a third gas clump moving again along (almost) the same orbital trace. Since the probability of finding three stars on close orbits is very small, this strongly argues against stellar-based source models. Instead, we show that the gas streamer G1-2-3 plausibly originates from the stellar wind of the massive binary star IRS16SW. This claim is substantiated by the fact that the small differences between the three orbits - the orientations of the orbital ellipses in their common plane as a function of time - are consistent with the orbital motion of IRS 16SW.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.00102/./TOI-4507_RV.png', 'tmp_2510.00102/./TOI-4507_LC.png', 'tmp_2510.00102/./SOAR.png', 'tmp_2510.00102/./Prot.png']
copying  tmp_2510.00102/./TOI-4507_RV.png to _build/html/
copying  tmp_2510.00102/./TOI-4507_LC.png to _build/html/
copying  tmp_2510.00102/./SOAR.png to _build/html/
copying  tmp_2510.00102/./Prot.png to _build/html/
exported in  _build/html/2510.00102.md
    + _build/html/tmp_2510.00102/./TOI-4507_RV.png
    + _build/html/tmp_2510.00102/./TOI-4507_LC.png
    + _build/html/tmp_2510.00102/./SOAR.png
    + _build/html/tmp_2510.00102/./Prot.png
found figures ['tmp_2510.00103/./figures/GN-9771_Ha_multiplicative_fit.png', 'tmp_2510.00103/./figures/BHS_line_emissivity_n10.png', 'tmp_2510.00103/./figures/GN-9771_Hb_multiplicative_fit.png']
copying  tmp_2510.00103/./figures/GN-9771_Ha_multiplicative_fit.png to _build/html/
copying  tmp_2510.00103/./figures/BHS_line_emissivity_n10.png to _build/html/
copying  tmp_2510.00103/./figures/GN-9771_Hb_multip

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\red}{\color{red}}$</div>



<div id="title">

# A Cold and Super-Puffy Planet on a Polar Orbit

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.00102-b31b1b.svg)](https://arxiv.org/abs/2510.00102)<mark>Appeared on: 2025-10-02</mark> -  _16 pages, 4 figures, submitted to AAS journals_

</div>
<div id="authors">

J. I. Espinoza-Retamal, et al. -- incl., <mark>T. Henning</mark>, <mark>J. Eberhardt</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of TOI-4507 b, a transiting sub-Saturn with a density $<$ 0.3 g/cm $^3$ on a 105-day polar orbit around a 700 Myr old F star. The transits were detected using datafrom TESS as well as the Antarctic telescope ASTEP. A joint analysis of the light curves and radial velocities from HARPS, FEROS, and CORALIE confirmed the planetary nature of the signal by limitingthe mass to be below $30 M_\oplus$ at 95 \% confidence. The radial velocities alsoexhibit the Rossiter-McLaughlin effectand imply that the star's equatorial planeis tiltedby $82.0_{-2.4}^{+2.6}$ deg with respect to the planet'sorbital plane. With these characteristics, TOI-4507 bis one of longest-period planets forwhich the stellar obliquity has been measured,and is among the longest-period and youngest "super-puff" planetsyet discovered.

</div>

<div id="div_fig1">

<img src="tmp_2510.00102/./TOI-4507_RV.png" alt="Fig2.1" width="50%"/><img src="tmp_2510.00102/./TOI-4507_LC.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** Different observations of TOI-4507 along with the best model. All error bars include a white noise jitter term applied in quadrature to the RV and photometric data points. a) ESPRESSO observations of the RM effect. We show the best model as the red line, and the $1\sigma$, $2\sigma$, and $3\sigma$ models as the shaded areas. b) Phase-folded out-of-transit RVs along with the best model. c) RVs as a function of time showing the long-term trend detected at $5\sigma$. d-g) Different light curves along with the best model. Binned data are shown as solid black points.
    The data used to create this figure are available as the data behind the figure.  (*fig:TOI-4507*)

</div>
<div id="div_fig2">

<img src="tmp_2510.00102/./SOAR.png" alt="Fig1" width="100%"/>

**Figure 1. -** High-resolution imaging observations of TOI-4507 taken with the 4.1 m SOAR telescope. We show the $5\sigma$ detection sensitivity and speckle autocorrelation functions. No companions to TOI-4507 within $3^{\prime\prime}$ are found in these observations. (*fig:soar*)

</div>
<div id="div_fig3">

<img src="tmp_2510.00102/./Prot.png" alt="Fig3" width="100%"/>

**Figure 3. -** a) Lomb-Scargle periodogram of the 2-min cadence TESS light curve of TOI-4507. A false alarm probability (FAP) level of 1\% is marked with a red dashed line. The black dashed line marks the highest-power peak, indicating the most likely stellar rotation period of 1.7 days. Aliases of this period are also shown as faint black dashed lines. b) TESS light curve phased to the rotational period of 1.7 days. All the work done here made use of the 2-min cadence light curve, but for illustrative purposes only, the 30-min cadence TESS light curve is shown here in blue, with binned data in red.
    The amplitude of the seen variability is $\sim500$ ppm. (*fig:Prot*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.00102"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\lya}{{\text{Ly}\ensuremath{\alpha}}}$
$\newcommand{\Halpha}{{\text{H}\ensuremath{\alpha}}}$
$\newcommand{\Hbeta}{{\text{H}\ensuremath{\beta}}}$
$\newcommand{\Hgamma}{{\text{H}\ensuremath{\gamma}}}$
$\newcommand{\oiii}{{\text{[\ion{O}{iii}]}}}$
$\newcommand{\niv}{{\text{\ion{N}{iv}]}}}$
$\newcommand{\feii}{{\text{\ion{Fe}{ii}}}}$
$\newcommand{\fesclya}{\ensuremath{f_{\rm esc}(\lya )}}$
$\newcommand{\fesclyc}{\ensuremath{f_{\rm esc}(\rm LyC)}}$
$\newcommand{\fcov}{\ensuremath{f_{\rm cov}}}$
$\newcommand{\jwst}{{JWST}}$
$\newcommand{\nircam}{NIRCam}$
$\newcommand{\muse}{MUSE}$
$\newcommand{\flambdaunits}{\ensuremath{\rm erg s^{-1} cm^{-2} {Å }^{-1}}}$
$\newcommand{\contblue}{{\it cont. blue}}$
$\newcommand{\contred}{{\it cont. red}}$
$\newcommand{\Hden}{\ensuremath{n_{\rm H}}}$
$\newcommand{\NH}{\ensuremath{N_{\rm H}}}$
$\newcommand{\NHI}{\ensuremath{N_{\rm \ion{H}{i}}}}$
$\newcommand{\eden}{\ensuremath{n_{\rm e}}}$
$\newcommand{\Ne}{\ensuremath{N_{\rm e}}}$
$\newcommand{\Te}{\ensuremath{T_{\rm e}}}$
$\newcommand{\logten}{\ensuremath{\log_{10}}}$
$\newcommand{\vturb}{\ensuremath{v_{\rm turb}}}$
$\newcommand{\logU}{\ensuremath{\logten U}}$
$\newcommand{\attnote}[1]{{\color{mycolor}[AT: #1]}}$</div>



<div id="title">

# The warm outer layer of a Little Red Dot as the source of [$\feii$ ] and collisional Balmer lines with scattering wings

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.00103-b31b1b.svg)](https://arxiv.org/abs/2510.00103)<mark>Appeared on: 2025-10-02</mark> -  _18 pages incl. references. Main figure: 4 ([Fe II] model). To be submitted to A&A. Comments welcome_

</div>
<div id="authors">

A. Torralba, et al. -- incl., <mark>A. d. Graaff</mark>

</div>
<div id="abstract">

**Abstract:** The population of the Little Red Dots (LRDs) may represent a key phase of supermassive black hole (SMBH) growth. A cocoon of dense excited gas is emerging as key component to explain the most striking properties of LRDs, such as strong Balmer breaks and Balmer absorption, as well as the weak IR emission. To dissect the structure of LRDs, we analyze new deep JWST/NIRSpec PRISM and G395H spectra of FRESCO-GN-9771, one of the most luminous known LRDs at $z=5.5$ . These reveal a strong Balmer break, broad Balmer lines and very narrow [ O ${\sc iii}$ ] emission. We unveil a forest of optical [ Fe ${\sc ii}$ ] lines, which we argue is emerging from a dense ( $n_{\rm H}=10^{9-10}$ cm $^{-3}$ ) warm layer with electron temperature $T_{\rm e}\approx7000$ K. The broad wings of H $\alpha$ and H $\beta$ have an exponential profile due to electron scattering in this same layer. The high $\rm H\alpha:H\beta:H\gamma$ flux ratio of $\approx10.4:1:0.14$ is an indicator of collisional excitation and resonant scattering dominating the Balmer line emission. A narrow H $\gamma$ component, unseen in the other two Balmer lines due to outshining by the broad components, could trace the ISM of a normal host galaxy with a star formation rate $\sim5$ M $_{\odot}$ yr $^{-1}$ . The warm layer is mostly opaque to Balmer transitions, producing a characteristic P-Cygni profile in the line centers suggesting outflowing motions. This same layer is responsible for shaping the Balmer break. The broad-band spectrum can be reasonably matched by a simple photoionized slab model that dominates the $\lambda>1500$ ${Å}$ continuum and a low mass ( $\sim10^8$ M $_{\odot}$ ) galaxy that could explain the narrow [ O ${\sc iii}$ ] , with only subdominant contribution to the UV continuum. Our findings indicate that Balmer lines are not directly tracing gas kinematics near the SMBH and that the BH mass scale is likely much lower than virial indicators suggest.

</div>

<div id="div_fig1">

<img src="tmp_2510.00103/./figures/GN-9771_Ha_multiplicative_fit.png" alt="Fig1" width="100%"/>

**Figure 1. -** ** Triangular P-Cygni H$\pmb\alpha$ spectrum of GN-9771.** The continuum-subtracted H$\alpha$ spectrum based on the G395H data is shown in black, whereas the red line shows the best-fit combined model. Residuals to the model are shown in the middle panel.
    The fiducial fitting model is described in Sect. \ref{sec:balmer_O3}.
    We indicate the masked regions based on the locations of possible narrow [$\feii$] emission in [Véron-Cetty, Joly and Véron (2004)]()(teal) and $\ion${He}{i}$\lambda$6680 (purple). The [$\ion${N}{ii}] component is not shown due to its relative flux being negligible. The bottom panel shows the $\Halpha$  spectrum and best-fit after subtracting the exponential component to highlight the P-Cygni profile. (*fig:Ha_line_fit*)

</div>
<div id="div_fig2">

<img src="tmp_2510.00103/./figures/BHS_line_emissivity_n10.png" alt="Fig8" width="100%"/>

**Figure 8. -** ** Where do lines originate from?** The intrinsic emissivity of selected emission lines in the photoionized Cloudy model, for $\Hden = 10^{10}$ \unit{cm^{-3}}, and $\NH = 10^{25}$ \unit{cm^{-2}}. Emissivities are expressed in arbitrary units of energy per unit time per unit volume. The ionization parameter is $\logU = -3$. The Hydrogen Balmer and Helium lines are intrinsically strong in the inner layers due to recombination emission, although the opacity of $\Halpha$  and $\Hbeta$  is high leading to scattering effects. [$\feii$] is mostly emitted in a warm layer with $\Te=6000$--7500 K, which is shielded from the far-UV ionizing radiation. (*fig:BHS_line_emissivities*)

</div>
<div id="div_fig3">

<img src="tmp_2510.00103/./figures/GN-9771_Hb_multiplicative_fit.png" alt="Fig2" width="100%"/>

**Figure 2. -** {** The H$\pmb\beta$  spectrum of GN-9771**. A similar model set-up was used as for H$\alpha$ described in Fig. \ref{fig:Ha_line_fit}}. Teal regions highlight that are masked due to possible $\feii$  emission. The orange region indicates the masked $\oiii$  wavelengths. The bottom panel shows the $\Hbeta$  spectrum and best fit (red) after subtracting the exponential component to highlight the P-Cygni profile. Figure \ref{fig:Hb_fit_fixed_exp} shows a version of the same fit with the exponential scale fixed to that fitted for $\Halpha$ . (*fig:Hb_line_fit*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.00103"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.}$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# A planetary system with a sub-Neptune planet \ in the habitable zone of TOI-2093

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.00299-b31b1b.svg)](https://arxiv.org/abs/2510.00299)<mark>Appeared on: 2025-10-02</mark> -  _Accepted by A&A. 12 pages, 7 figures, 5 tables in the main body (20 pages, 19 figures, 9 tables including appendices)_

</div>
<div id="authors">

J. Sanz-Forcada, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:**            Aims. We aim to confirm and measure the mass of the transiting planet candidate around the K5V star TOI-2093, previously announced by the Transiting Exoplanet Survey Satellite (TESS) project. Methods. We combined photometric data from 32 sectors between 2019 and 2024 with 86 radial velocity measurements obtained with the CARMENES spectrograph over a period of 2.4 years, along with a series of ground-based, broadband photometric monitoring campaigns to characterize the host star and the transiting planet candidate, as well as to search for additional planets in the system. Our data indicate that TOI-2093 is a main-sequence star located at a distance of 83 pc, with solar metallicity, and a rotation period of 43.8 +- 1.8 d. Results. We have confirmed the planetary nature of the TESS transiting planet candidate, named TOI-2093 c, through the detection of its Keplerian signal in the spectroscopic data. We measured a planetary radius of 2.30 +- 0.12 Rearth, a Neptune-like mass of 15.8 +- 3.7 Mearth, and an orbital period of 53.81149 +- 0.00017 d. This makes TOI-2093 c the smallest exoplanet known in the habitable zone of a main-sequence FGK star. Given its size and relatively high density, TOI-2093 c belongs to a class of planets with no analog in the Solar System. In addition, the CARMENES data revealed the presence of a second planet candidate with a minimum mass of 10.6 +- 2.5 Mearth and an orbital period of 12.836 +- 0.021 d. This inner planet, which we designated TOI-2093 b, shows no detectable photometric transit in the TESS light curves. The orbital planes of the two planets are misaligned by more than 1.6 deg despite the near 4:1 mean-motion resonance of their orbital periods.         

</div>

<div id="div_fig1">

<img src="tmp_2510.00299/./Figs/GLS_activity_period_v3.png" alt="Fig1" width="100%"/>

**Figure 1. -** GLS periodograms of all photometric and spectroscopic data. Vertical dashed lines mark planetary orbital periods (transiting planet on a cyan band), and vertical green bands indicate stellar activity timescales (see text). Horizontal dashed lines show FAP levels (0.1 \% in green, 1 \% in orange, and 10 \% in blue) across all panels. (*fig:glsgeneral*)

</div>
<div id="div_fig2">

<img src="tmp_2510.00299/./Figs/GLS_output_graphic_window.png" alt="Fig8" width="100%"/>

**Figure 8. -** Window function of TOI-2093 CARMENES VIS RV data. (*fig:window_carmenes_vis*)

</div>
<div id="div_fig3">

<img src="tmp_2510.00299/./Figs/massrad.png" alt="Fig14.1" width="50%"/><img src="tmp_2510.00299/./Figs/teqvesc.png" alt="Fig14.2" width="50%"/>

**Figure 14. -** TOI 2093 c in the context of exoplanets with known mass and radius with errors smaller than 12\%. * Left panel:* Planetary mass and radius, with theoretical models from \citet{zen16}(see Sect. \ref{sec:TOI-2093 planetary system}). * Right panel:* Escape velocity against equilibrium temperature (planetary data from the NASA exoplanets database).
 TOI 2093 b is included assuming $i_{\rm b} \gtrsim 50◦$,
      with the planet radius calculated for a bulk density between 0.5 and 10 g cm$^{-3}$, with errors shown as a red band. (*fig:diagramaMR*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.00299"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\omc}{\omega Cen}$
$\newcommand{\ebv}{\ensuremath{E(B-V)}}$
$\newcommand{\nad}{\ensuremath{\mathrm{Na \textsc{i} D}}}$
$\newcommand{\nadtwo}{\ensuremath{\mathrm{Na \textsc{i} D_2}}}$
$\newcommand{\nadone}{\ensuremath{\mathrm{Na \textsc{i} D_1}}}$
$\newcommand{\snrnad}{\rm{S/N_{\ensuremath{\mathrm{Na \textsc{i} D}}}}}$
$\newcommand{\ewismdtwo}{\rm{EW}_{\rm{D2, ISM}}}$
$\newcommand{\ewismdone}{\rm{EW}_{\rm{D1, ISM}}}$
$\newcommand{\ewicmdtwo}{\rm{EW}_{\rm{D2, ICM}}}$
$\newcommand{\ewicmdone}{\rm{EW}_{\rm{D1, ICM}}}$
$\newcommand{\ewicmdoneul}{\rm{EW}_{\rm{D1, ICM}}^{\rm{upper}}}$
$\newcommand{\vlos}{V_{\rm LOS}}$
$\newcommand{\teff}{\ensuremath{T_{\mathrm{eff}}}}$
$\newcommand{\maspyr}{\rm{mas}~\rm{yr}^{-1}}$
$\newcommand{\perpixel}{\text{pix}^{-1}}$
$\newcommand{\maxperyr}{\rm{mas}~\rm{yr}^{-1}}$
$\newcommand{\angstrom}{\mbox{\normalfontÅ}}$
$\newcommand{\kms}{\rm{km~s^{-1}}}$</div>



<div id="title">

# oMEGACat. VII. Tracing Interstellar and Intracluster Medium of $\omega$ Centauri using Sodium Absorptions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.00330-b31b1b.svg)](https://arxiv.org/abs/2510.00330)<mark>Appeared on: 2025-10-02</mark> -  _23 pages, 11 figures, and 2 tables, accepted by ApJ. Machine-readable data is available in the online article_

</div>
<div id="authors">

Z. Wang (王梓先), et al. -- incl., <mark>M. Häberle</mark>, <mark>N. Neumayer</mark>, <mark>C. Clontz</mark>

</div>
<div id="abstract">

**Abstract:** We investigate the foreground interstellar medium along the line of sight and intracluster medium of $\omega$ Centauri ( $\omc$ ) by measuring the equivalent width of $\nad$ absorptions from MUSE observations.The large line-of-sight velocity difference between $\omc$ and the foreground enables us to separate $\nad$ absorption contributed from atomic gas in the interstellar and intracluster medium.We find that small-scale substructures in the foreground $\nad$ distribution correlate with differential reddening derived from photometric methods.Using an empirical $\nad$ equivalent width-reddening relation, we determine an average reddening of $E(B-V)=0.153\pm0.003$ mag within the half-light radius of $\omc$ .However, the $\nad$ -inferred differential reddening is significantly larger than photometric estimates. This is likely due to scatter in the $\nad$ -reddening relation.We find no evidence for intracluster atomic gas from spectra of horizontal branch stars, as there is no significant $\nad$ absorption at $\omc$ 's systemic velocity.Given this non-detection, we place the strongest upper limit to date on the intracluster atomic gas column density in $\omc$ of $\lesssim2.17 \times 10^{18} \rm{cm^{-2}}$ .We also estimate the ionized gas density from pulsar dispersion measure variations, which exceed the atomic gas limit by $\sim$ 50 times.Nevertheless, the strong correlation between dispersion measure and foreground $\nad$ suggests that much or all of this ionized gas resides in the foreground.Given ongoing mass loss from bright giant stars, our findings imply that the intracluster gas accumulation timescale is short, and gas removal in the cluster is likely not tied to stripping as $\omc$ passes through the Galactic disk.

</div>

<div id="div_fig1">

<img src="tmp_2510.00330/./CMD_rv_pm_samplev5.png" alt="Fig8" width="100%"/>

**Figure 8. -** 
Data sample used in this study.
**Panel (a):** CMD of the data sample in Section \ref{subsec:data-muse-hst} using \citetalias{2010ApJ...710.1032A} photometry.
The blue region defines the selection of warm HB stars.
The black dotted line shows the ZAHB track from the BaSTI model (see Section \ref{subsubsec:data-target-selection-hb} for details).
**Panel (b):**$\vlos$  distribution of the warm HB stars and all other stars. The gray histogram indicates stars excluded based on the selection criteria for the two samples.
**Panel (c):** Proper motion distribution of the same samples, with colors corresponding to panel (b).
**Panel (d):** Re-estimated S/N distribution in the $\nad$  region.
**Panel (e) and (f):** Example MUSE spectra (black line) of warm HB and all other stars, with the S/N and $\vlos$  indicated in the top left corner.
The green lines show the residuals between the observed spectra and the best-fit SPEXXY model from \citetalias{2023ApJ...958....8N}.
Vertical lines are the positions of the $\nad$  absorptions from foreground (solid, located at 0 $\kms$) and intracluster (dashed, at 232.7 $\kms$) atomic gas of $\omc$ .
An additional contribution from the stellar photosphere, seen as the right-most absorption feature, is present in panel (f).
 (*fig:samplefig*)

</div>
<div id="div_fig2">

<img src="tmp_2510.00330/./fitting_examplev2.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
    Demonstration of spectral fitting to measure the EWs of $\nad$ , as described in Section \ref{subsec:methods-ew}.
    The top panel shows an example spectrum of a warm HB star, while the bottom panel shows a Voronoi-binned residual spectrum from all other stars.
    The equivalent widths (EWs) of the $\nad$  lines, in units of Å , are labeled in each panel.
    The red-shaded region indicates the wavelength range used for fitting.
    The gray-shaded regions are used to estimate the continuum ($f_{c}$) in Eq. \ref{eqn:nad-fit}.
    Fitting residuals are shown in green as a separate sub-panel.
    The warm HB spectrum is expected to have minimal photospheric contribution to $\nad$  and thus we name them as "ISM" and "ICM".
    For all other stars that are mostly cooler, contamination from the stellar photosphere could remain in the SPEXXY residuals.
    Therefore, we label the absorptions at $\omc$  systemic velocity as "others".
    Note that the D2 line is at a shorter wavelength than the D1 line for both the ISM and ICM/others components.
     (*fig:fitting-example*)

</div>
<div id="div_fig3">

<img src="tmp_2510.00330/./ew_2d_distribution.png" alt="Fig9" width="100%"/>

**Figure 9. -** 
    Spatial distribution of $\ewismdtwo$  from the foreground Na atomic gas.
    The left panel shows individual measurements from the original spectra of warm HB stars.
    The right panel shows measurements from Voronoi-binned SPEXXY residual spectra \citepalias{2023ApJ...958....8N} of all other stars.
    The number of stars used for each panel is noted in the subtitles.
    The x- and y-axes indicate coordinates in $(\alpha, \delta)$ relative to the center of $\omc$ .
    The two blank regions near the center of $\omc$  are due to the lack of non-AO observations.
    Both warm HB and all other stars demonstrate clear spatial substructures of foreground Na atomic gas, and they are consistent with the differential reddening distribution from previous studies (see Section \ref{subsec:discuss-comparison} for detailed discussion).
    The individual warm HB $\ewismdtwo$  measurements and the Voronoi-binned $\ewismdtwo$  map with uncertainties are available as a machine-readable table (columns listed in Table. \ref{tab:ewd2ism_table}) and a FITS image, respectively.
     (*fig:ew-ism-distribution*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.00330"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Muv}{M_{\rm{UV}}}$
$\newcommand{\spitzer}{\textit{Spitzer}/IRAC\xspace}$
$\newcommand{\zphot}{z_{\rm{phot}}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$: Discovery of bright $\boldsymbol{z\simeq7}$ Lyman-break galaxies in UltraVISTA and $\Euclid$ COSMOS$\thanks{This paper is published on       behalf of the Euclid Consortium}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.00945-b31b1b.svg)](https://arxiv.org/abs/2510.00945)<mark>Appeared on: 2025-10-02</mark> -  _18 pages, 13 figures, 5 tables, additional figures in appendix. Submitted to A&A_

</div>
<div id="authors">

R. G. Varadaraj, et al. -- incl., <mark>E. Bañados</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We present a search for $z\simeq7$ Lyman-break galaxies using the $1.72   \rm{deg}^2$ near-infrared UltraVISTA survey in the COSMOS field, reaching $5 \sigma$ depths in $Y$ of 26.2.We incorporate deep optical and _Spitzer_ imaging for a full spectral energy distribution (SED) fitting analysis.We find 289 candidate galaxies at $6.5\leq z \leq 7.5$ covering $-22.6 \leq \Muv \leq -20.2$ , faint enough to overlap with _Hubble_ Space Telescope studies.We conduct a separate selection by including complementary $\Euclid$ performance verification imaging (reaching $5 \sigma$ depths of $26.3$ ), yielding 140 galaxies in $0.65   \rm{deg}^2$ , with 38 sources unique to this sample.We compute the rest-frame UV luminosity function (UV LF) from our samples, extending below the knee ( $M^*=-21.14^{+0.28}_{-0.25}$ ).We find that the shape of the UV LF is consistent with both a Schechter function and double-power law (DPL) at the magnitudes probed by this sample, with a DPL preferred at $\Muv<-22.5$ when bright-end results are included.The UltraVISTA+ $\Euclid$ sample provides a clean measurement of the LF due to the overlapping near-infrared filters identifying molecular absorption features in the SEDs of ultra-cool dwarf interlopers, and additional faint galaxies are recovered.A comparison with JWST LFs at $z>7$ suggests a gentle evolution in the bright-end slope, although this is limited by a lack of robust bright-end measurements at $z>9$ .We forecast that in the Euclid Deep Fields, the removal of contaminant ultra-cool dwarfs as point sources will be possible at $\JE < 24.5$ .Finally, we present a high-equivalent-width Lyman- $\alpha$ emitter candidate identified by combining HSC, VISTA, and $\Euclid$ broadband photometry, highlighting the synergistic power these instruments will have in the Euclid Auxiliary Fields for identifying extreme sources in the Epoch of Reionisation.

</div>

<div id="div_fig1">

<img src="tmp_2510.00945/./figures/depths_and_model_mags.png" alt="Fig12" width="100%"/>

**Figure 12. -** Limiting magnitudes, or modal depths ($5 \sigma$) of the photometric filters used in this work within the COSMOS field.
    The line widths represent the FWHM of the filter transmission curves, and the depths are reported in Table \ref{tab:Depths}.
    The HSC and VISTA filters are shown in black and orange, respectively, and the four \Euclid filters are labelled.
    We also show example SEDs of a UCD and a LBG with the solid and dashed lines, respectively.
    The model photometry for the UCD and the LBG are shown by the stars and circles respectively, and are colour-coded by their filters.
    Note that the \Euclid NIR filters cover the gaps between the VISTA filters. These wavelengths are inaccessible from the ground due to the atmospheric absorption. (*fig:filters*)

</div>
<div id="div_fig2">

<img src="tmp_2510.00945/./figures/381772_no_euclid.png" alt="Fig13.1" width="50%"/><img src="tmp_2510.00945/./figures/381772_with_euclid.png" alt="Fig13.2" width="50%"/>

**Figure 13. -** 
    SED fitting of a candidate LBG, EUCL J100041.40$+$020157.5 (hereafter LBG 10004$+$02015), at $z=6.90$.
    Additional candidates are presented in Appendix \ref{sec: candidate seds and stamps}.
    *Left*: the SED fitting without \Euclid data, as part of the U-only selection (see Sect. \ref{sec: candidate selection}).
    *Right*: the SED fitting including \Euclid data, as part of the U+E selection.
    The HSC, VISTA and \Euclid photometry are shown by the points, diamonds and squares, respectively.
    The photometry is coloured following Fig. \ref{fig:filters}.
    Non-detections are replaced with $2 \sigma$ upper limits.
    We also show the filter widths for VISTA and \Euclid at the top of the plot.
    The $\spitzer$ model photometry and upper limit in the $3.6 \micron$ filter is also shown.
    The blue curve shows the best high-redshift solution, and the grey open circles are its expected model photometry.
    The orange curve shows the best dusty low-redshift solution, and the red curve shows the best UCD solution.
    The legend in the top right shows the redshift and $\chi^2$ of the galaxy solutions, and the $\chi^2$ and spectral type of the UCD solution.
    The inset panel shows the redshift probability distribution for this source.
    The inclusion of \Euclid data reinforces the exclusion the UCD solution.
    Also note that the inclusion of \Euclid data prefers a bluer slope. (*fig:UVISTA euclid SED*)

</div>
<div id="div_fig3">

<img src="tmp_2510.00945/./figures/z_Muv_sample_marg.png" alt="Fig2.1" width="50%"/><img src="tmp_2510.00945/./figures/z_Muv_sample_with_euclid_marg.png" alt="Fig2.2" width="50%"/>

**Figure 2. -** The $z\simeq7$ LBG sample from this work plotted in photometric redshift $z_{\rm{phot}}$ and absolute rest-frame UV magnitude $\Muv$ space.
    *Top*: the U-only sample (blue).
    *Bottom*: the U+E sample (red).
    The red squares with a black outline indicate galaxies which are not recovered in the U-only sample, and are thus unique to the U+E sample.
    In both panels, we also show the candidates from \citetalias{Varadaraj23} in the XMM-LSS and ECDF-S fields, and the HST-selected candidates from \citetalias{Bouwens21}.
    The dashed grey lines show the 30\%, 40\%, and 50\% completeness limits, as derived from the injection-recovery simulation (see Sect. \ref{sec:completeness}).
    We also show the marginalised distributions in $z_{\rm{phot}}$ and $\Muv$ as normalised histograms, and also overplot the distribution of the other sample as the thinner, fainter line for comparison. The mean uncertainties are shown on the bottom left. (*fig:z-Muv*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.00945"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

130  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

18  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

10  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
